# Bias-Variance Tradeoff

## Введение

Bias-Variance Tradeoff — это фундаментальная концепция в машинном обучении, объясняющая компромисс между двумя источниками ошибок модели:

- **Bias (смещение)** — систематическая ошибка модели, неспособность уловить истинную зависимость
- **Variance (разброс)** — чувствительность модели к вариациям в обучающих данных

### Применение в биологии:
- Предсказание структуры белков
- Анализ экспрессии генов
- Классификация типов опухолей
- Моделирование взаимодействий лекарств

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

np.random.seed(42)

## 1. Математические основы

Ожидаемая ошибка модели разлагается на три компоненты:

$$\text{Expected Error} = \text{Bias}^2 + \text{Variance} + \text{Irreducible Error}$$

где:
- **Bias²** — $(\mathbb{E}[\hat{f}(x)] - f(x))^2$ — систематическая ошибка
- **Variance** — $\mathbb{E}[(\hat{f}(x) - \mathbb{E}[\hat{f}(x)])^2]$ — вариабельность предсказаний
- **Irreducible Error** — шум в данных, который невозможно устранить

## 2. Визуализация концепции

In [ ]:
# Создаем истинную функцию (нелинейная зависимость)
def true_function(x):
    """Истинная зависимость: концентрация метаболита от времени"""
    return np.sin(1.5 * x) + 0.5 * x

# Генерируем данные
n_samples = 50
X_train = np.linspace(0, 5, n_samples)
y_true = true_function(X_train)
noise = np.random.normal(0, 0.3, n_samples)
y_train = y_true + noise

# Тестовая выборка (без шума)
X_test = np.linspace(0, 5, 200)
y_test_true = true_function(X_test)

print(f"Размер обучающей выборки: {n_samples}")
print(f"Стандартное отклонение шума: 0.3")

In [ ]:
# Обучаем модели с разной сложностью
degrees = [1, 3, 15]  # степени полинома
models = {}

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, degree in enumerate(degrees):
    # Полиномиальные признаки
    poly = PolynomialFeatures(degree=degree)
    X_train_poly = poly.fit_transform(X_train.reshape(-1, 1))
    X_test_poly = poly.transform(X_test.reshape(-1, 1))
    
    # Обучаем модель
    model = LinearRegression()
    model.fit(X_train_poly, y_train)
    y_pred = model.predict(X_test_poly)
    
    # Вычисляем ошибку
    mse = mean_squared_error(y_test_true, y_pred)
    
    # Визуализация
    axes[idx].scatter(X_train, y_train, alpha=0.6, s=50, label='Обучающие данные')
    axes[idx].plot(X_test, y_test_true, 'g-', linewidth=2, label='Истинная функция')
    axes[idx].plot(X_test, y_pred, 'r--', linewidth=2, label=f'Модель (степень {degree})')
    axes[idx].set_xlabel('Время (часы)')
    axes[idx].set_ylabel('Концентрация метаболита')
    axes[idx].set_title(f'Степень {degree}\nMSE: {mse:.3f}')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)
    
    if degree == 1:
        axes[idx].text(0.5, -1, 'HIGH BIAS\nLOW VARIANCE', 
                      bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5),
                      fontsize=10, ha='center')
    elif degree == 15:
        axes[idx].text(0.5, -1, 'LOW BIAS\nHIGH VARIANCE', 
                      bbox=dict(boxstyle='round', facecolor='orange', alpha=0.5),
                      fontsize=10, ha='center')
    else:
        axes[idx].text(0.5, -1, 'BALANCED', 
                      bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5),
                      fontsize=10, ha='center')

plt.tight_layout()
plt.show()

## 3. Эмпирическая оценка Bias и Variance

Создадим множество обучающих выборок и оценим bias и variance.

In [ ]:
def compute_bias_variance(degree, n_datasets=100):
    """Вычисление bias и variance для модели заданной сложности"""
    
    # Тестовая точка
    X_test_point = np.array([[2.5]])  # середина диапазона
    y_true_point = true_function(X_test_point[0, 0])
    
    predictions = []
    
    # Генерируем множество датасетов
    for _ in range(n_datasets):
        # Новая выборка
        X_sample = np.linspace(0, 5, 30)
        y_sample = true_function(X_sample) + np.random.normal(0, 0.3, 30)
        
        # Обучаем модель
        poly = PolynomialFeatures(degree=degree)
        X_sample_poly = poly.fit_transform(X_sample.reshape(-1, 1))
        X_test_poly = poly.transform(X_test_point)
        
        model = LinearRegression()
        model.fit(X_sample_poly, y_sample)
        
        # Предсказание
        y_pred = model.predict(X_test_poly)[0]
        predictions.append(y_pred)
    
    predictions = np.array(predictions)
    
    # Вычисляем bias и variance
    bias_squared = (np.mean(predictions) - y_true_point) ** 2
    variance = np.var(predictions)
    
    return bias_squared, variance

# Вычисляем для разных степеней полинома
degrees_range = range(1, 16)
biases = []
variances = []
total_errors = []

for degree in degrees_range:
    bias_sq, var = compute_bias_variance(degree)
    biases.append(bias_sq)
    variances.append(var)
    total_errors.append(bias_sq + var)
    
print("Вычисление завершено!")

In [ ]:
# Визуализация Bias-Variance Tradeoff
plt.figure(figsize=(12, 6))

plt.plot(degrees_range, biases, 'b-o', linewidth=2, markersize=6, label='Bias²')
plt.plot(degrees_range, variances, 'r-s', linewidth=2, markersize=6, label='Variance')
plt.plot(degrees_range, total_errors, 'g-^', linewidth=2, markersize=6, label='Total Error (Bias² + Variance)')

# Находим оптимальную степень
optimal_degree = degrees_range[np.argmin(total_errors)]
plt.axvline(optimal_degree, color='purple', linestyle='--', linewidth=2, 
            label=f'Оптимальная сложность (степень {optimal_degree})')

plt.xlabel('Сложность модели (степень полинома)')
plt.ylabel('Ошибка')
plt.title('Bias-Variance Tradeoff')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')

# Аннотации
plt.annotate('Underfitting\n(High Bias)', xy=(2, 0.1), xytext=(3, 0.5),
            arrowprops=dict(arrowstyle='->', color='blue', lw=1.5),
            fontsize=11, color='blue')
plt.annotate('Overfitting\n(High Variance)', xy=(14, 1), xytext=(12, 2),
            arrowprops=dict(arrowstyle='->', color='red', lw=1.5),
            fontsize=11, color='red')

plt.tight_layout()
plt.show()

print(f"\nОптимальная степень полинома: {optimal_degree}")
print(f"Минимальная ошибка: {min(total_errors):.4f}")

## 4. Влияние размера выборки

In [ ]:
def learning_curve(degree, sample_sizes):
    """Кривые обучения для разных размеров выборки"""
    train_errors = []
    test_errors = []
    
    X_test_full = np.linspace(0, 5, 200).reshape(-1, 1)
    y_test_full = true_function(X_test_full.ravel())
    
    for n in sample_sizes:
        # Генерируем выборку
        X = np.linspace(0, 5, n).reshape(-1, 1)
        y = true_function(X.ravel()) + np.random.normal(0, 0.3, n)
        
        # Обучаем модель
        poly = PolynomialFeatures(degree=degree)
        X_poly = poly.fit_transform(X)
        X_test_poly = poly.transform(X_test_full)
        
        model = LinearRegression()
        model.fit(X_poly, y)
        
        # Ошибки
        train_pred = model.predict(X_poly)
        test_pred = model.predict(X_test_poly)
        
        train_errors.append(mean_squared_error(y, train_pred))
        test_errors.append(mean_squared_error(y_test_full, test_pred))
    
    return train_errors, test_errors

# Тестируем для разных сложностей модели
sample_sizes = [10, 20, 30, 50, 75, 100, 150, 200]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, degree in enumerate([1, 5, 15]):
    train_errors, test_errors = learning_curve(degree, sample_sizes)
    
    axes[idx].plot(sample_sizes, train_errors, 'b-o', linewidth=2, label='Ошибка на обучении')
    axes[idx].plot(sample_sizes, test_errors, 'r-s', linewidth=2, label='Ошибка на тесте')
    axes[idx].set_xlabel('Размер обучающей выборки')
    axes[idx].set_ylabel('MSE')
    axes[idx].set_title(f'Learning Curve (степень {degree})')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)
    axes[idx].set_ylim([0, max(max(test_errors), 2)])

plt.tight_layout()
plt.show()

## 5. Регуляризация как инструмент контроля Bias-Variance

Регуляризация помогает найти баланс между bias и variance, штрафуя сложность модели.

In [ ]:
# Генерируем данные
n_samples = 50
X = np.linspace(0, 5, n_samples).reshape(-1, 1)
y = true_function(X.ravel()) + np.random.normal(0, 0.3, n_samples)

X_test_reg = np.linspace(0, 5, 200).reshape(-1, 1)
y_test_reg = true_function(X_test_reg.ravel())

# Высокая степень полинома с разной регуляризацией
degree = 15
poly = PolynomialFeatures(degree=degree)
X_poly = poly.fit_transform(X)
X_test_poly = poly.transform(X_test_reg)

# Разные значения регуляризации
alphas = [0, 0.01, 1, 100]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

for idx, alpha in enumerate(alphas):
    if alpha == 0:
        model = LinearRegression()
        title = 'Без регуляризации'
    else:
        model = Ridge(alpha=alpha)
        title = f'Ridge регуляризация (α={alpha})'
    
    model.fit(X_poly, y)
    y_pred = model.predict(X_test_poly)
    mse = mean_squared_error(y_test_reg, y_pred)
    
    axes[idx].scatter(X, y, alpha=0.6, s=50, label='Данные')
    axes[idx].plot(X_test_reg, y_test_reg, 'g-', linewidth=2, label='Истинная функция')
    axes[idx].plot(X_test_reg, y_pred, 'r--', linewidth=2, label='Предсказания')
    axes[idx].set_xlabel('X')
    axes[idx].set_ylabel('y')
    axes[idx].set_title(f'{title}\nMSE: {mse:.3f}')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)
    axes[idx].set_ylim([-3, 4])

plt.tight_layout()
plt.show()

## 6. Биологический пример: Предсказание активности лекарства

Предсказываем эффективность лекарства на основе молекулярных дескрипторов.

In [ ]:
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score

# Генерируем данные: молекулярные дескрипторы -> активность лекарства
X_drug, y_drug = make_regression(
    n_samples=200,
    n_features=20,
    n_informative=10,
    noise=10,
    random_state=42
)

X_train_drug, X_test_drug, y_train_drug, y_test_drug = train_test_split(
    X_drug, y_drug, test_size=0.3, random_state=42
)

print(f"Размер обучающей выборки: {len(X_train_drug)}")
print(f"Количество признаков: {X_train_drug.shape[1]}")
print(f"Размер тестовой выборки: {len(X_test_drug)}")

In [ ]:
# Сравниваем модели с разной сложностью
from sklearn.linear_model import Lasso

models = {
    'Linear Regression\n(Low Bias, High Variance)': LinearRegression(),
    'Ridge (α=1)\n(Balanced)': Ridge(alpha=1),
    'Ridge (α=100)\n(High Bias, Low Variance)': Ridge(alpha=100),
    'Decision Tree (depth=10)\n(Low Bias, High Variance)': DecisionTreeRegressor(max_depth=10, random_state=42),
    'Decision Tree (depth=3)\n(High Bias, Low Variance)': DecisionTreeRegressor(max_depth=3, random_state=42),
    'Random Forest\n(Balanced)': RandomForestRegressor(n_estimators=50, max_depth=5, random_state=42)
}

results = []

for name, model in models.items():
    # Cross-validation
    cv_scores = cross_val_score(model, X_train_drug, y_train_drug, cv=5, 
                                 scoring='neg_mean_squared_error')
    cv_mse = -cv_scores.mean()
    cv_std = cv_scores.std()
    
    # Обучение и тест
    model.fit(X_train_drug, y_train_drug)
    train_mse = mean_squared_error(y_train_drug, model.predict(X_train_drug))
    test_mse = mean_squared_error(y_test_drug, model.predict(X_test_drug))
    
    results.append({
        'Model': name,
        'Train MSE': train_mse,
        'CV MSE': cv_mse,
        'Test MSE': test_mse,
        'Overfit': test_mse - train_mse
    })

# Визуализация результатов
import pandas as pd

df_results = pd.DataFrame(results)
print("\nСравнение моделей:")
print(df_results.to_string(index=False))

In [ ]:
# Визуализация сравнения
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# График 1: Train vs Test MSE
x_pos = np.arange(len(df_results))
width = 0.35

axes[0].bar(x_pos - width/2, df_results['Train MSE'], width, label='Train MSE', alpha=0.8)
axes[0].bar(x_pos + width/2, df_results['Test MSE'], width, label='Test MSE', alpha=0.8)
axes[0].set_xlabel('Модель')
axes[0].set_ylabel('MSE')
axes[0].set_title('Сравнение ошибок на обучении и тесте')
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels([name.split('\n')[0] for name in df_results['Model']], 
                        rotation=45, ha='right')
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# График 2: Степень переобучения
colors = ['green' if x < 500 else 'orange' if x < 1000 else 'red' for x in df_results['Overfit']]
axes[1].bar(x_pos, df_results['Overfit'], color=colors, alpha=0.7)
axes[1].axhline(0, color='black', linestyle='--', linewidth=1)
axes[1].set_xlabel('Модель')
axes[1].set_ylabel('Overfit (Test MSE - Train MSE)')
axes[1].set_title('Степень переобучения')
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels([name.split('\n')[0] for name in df_results['Model']], 
                        rotation=45, ha='right')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 7. Практические рекомендации

### Признаки High Bias (Underfitting):
- Высокая ошибка на обучающей выборке
- Высокая ошибка на тестовой выборке
- Малая разница между ошибками

**Решения:**
- Увеличить сложность модели
- Добавить больше признаков
- Уменьшить регуляризацию

### Признаки High Variance (Overfitting):
- Низкая ошибка на обучающей выборке
- Высокая ошибка на тестовой выборке
- Большая разница между ошибками

**Решения:**
- Собрать больше данных
- Уменьшить сложность модели
- Использовать регуляризацию
- Применить кросс-валидацию

## 8. Визуализация концепции: мишень

In [ ]:
# Классическая визуализация Bias-Variance через мишень
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

scenarios = [
    {'title': 'Low Bias, Low Variance\n(Идеальная модель)', 'bias': 0, 'variance': 0.5, 'pos': (0, 0)},
    {'title': 'Low Bias, High Variance\n(Overfitting)', 'bias': 0, 'variance': 3, 'pos': (0, 1)},
    {'title': 'High Bias, Low Variance\n(Underfitting)', 'bias': 4, 'variance': 0.5, 'pos': (1, 0)},
    {'title': 'High Bias, High Variance\n(Худший случай)', 'bias': 4, 'variance': 3, 'pos': (1, 1)}
]

for scenario in scenarios:
    ax = axes[scenario['pos']]
    
    # Рисуем мишень
    circles = [plt.Circle((0, 0), r, fill=False, color='black', linewidth=2) 
               for r in [2, 4, 6, 8]]
    for circle in circles:
        ax.add_patch(circle)
    
    # Центр мишени
    ax.plot(0, 0, 'r*', markersize=20, label='Истинное значение')
    
    # Генерируем выстрелы (предсказания модели)
    n_shots = 20
    shots_x = np.random.normal(scenario['bias'], scenario['variance'], n_shots)
    shots_y = np.random.normal(scenario['bias'], scenario['variance'], n_shots)
    
    ax.scatter(shots_x, shots_y, c='blue', s=100, alpha=0.6, label='Предсказания')
    
    # Среднее предсказаний
    ax.plot(np.mean(shots_x), np.mean(shots_y), 'go', markersize=15, 
            label='Среднее предсказание', markeredgecolor='black', markeredgewidth=2)
    
    ax.set_xlim(-10, 10)
    ax.set_ylim(-10, 10)
    ax.set_aspect('equal')
    ax.set_title(scenario['title'], fontsize=12, fontweight='bold')
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)
    ax.axhline(0, color='gray', linestyle='--', linewidth=0.5)
    ax.axvline(0, color='gray', linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.show()

## 9. Задания для самостоятельной работы

1. Исследуйте влияние разных типов регуляризации (L1, L2, ElasticNet) на bias-variance tradeoff
2. Реализуйте bootstrap для оценки variance модели
3. Примените концепции к реальным биологическим данным (например, из UCI ML Repository)
4. Сравните bias-variance tradeoff для различных алгоритмов (SVM, KNN, нейронные сети)